# Weekly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import psycopg2
import pandas as pd
import matplotlib
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from IPython.core.display import HTML

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
connect_to_db = 'postgresql://' + \
                os.getenv('PG_USER', '') + ":" + os.getenv('PG_PASSWORD', '') +'@' + \
                os.getenv('PG_HOST', '') + ':' + os.getenv('PG_PORT', '5432') + '/' + os.getenv('PG_DB_NAME', '');
    
%sql $connect_to_db

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

Weekly running time.

In [ ]:
%%sql stat_weekly_restoration  <<
SELECT r.nr_num, n.name, r.submitted_date, r.expiration_date, n.state 
FROM requests r, names n, events e
WHERE r.id=n.nr_id
AND r.id=e.nr_id
AND r.request_type_cd in ('RCR', 'RUL', 'BERE', 'RCC', 'RCP', 'RFI', 'XRCR', 'RLC', 'XRCP')
AND n.state in ('APPROVED', 'CONDITION')
AND e.state_cd in ('APPROVED', 'CONDITIONAL')
AND e.action='nro_update' 
AND date(e.event_dt at time zone 'utc' at time zone 'pst') >= date(current_date-7)
AND date(e.event_dt at time zone 'utc' at time zone 'pst') <= date(current_date-1)
GROUP BY r.nr_num, n.name, r.submitted_date, r.expiration_date, n.state
ORDER BY r.expiration_date desc;

In [ ]:
filename = os.getenv('DATA_DIR', '')+'weekly_restoration_expiry_till_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m-%d') +'.csv'
restoration_edt=stat_weekly_restoration.DataFrame()

with open(filename, 'w') as f:
    if restoration_edt.empty:
        f.write('No Data Retrieved')  
    else:    
        restoration_edt.to_csv(f, sep=',', encoding='utf-8', index=False)